In [2]:
import pandas as pd

## TESTS for indicators_transformer.py script

In [2]:
# Let's convert pivot table into a reguilar table for all the interesting indicators

filenames = [
    "ecosystems.csv", "habitat.csv", "infrastructure.csv"   # Vulnerability
    "economic.csv", "governance.csv", "social.csv"          # Readingess
    ]

In [35]:
filename = "ecosystems.csv"
df = pd.read_csv(f"../../data/resources/vulnerability/{filename}")


In [22]:
melted_df = df.melt(
    id_vars=["ISO3", "Name"],
    var_name="Year",
    value_name="value"
)

In [24]:
melted_df["no_value_ecosystem"] = melted_df["value"].isna()

In [37]:
# Get mean by year and replace nan values by mean
mean_df = melted_df.groupby(["Year"])[["value"]].mean().reset_index()
mean_df.rename(columns={"value": "mean"}, inplace=True)
final_df = pd.merge(melted_df, mean_df, how="left", on="Year")
final_df.loc[final_df["value"].isna(), "value"] = final_df["mean"]
del final_df["mean"]
# Save result to CSV
melted_filename = f"melted_{filename}"
final_df.to_csv(f"../../data/resources/vulnerability/{melted_filename}", index=False)


## EXTRA: Let's convert governance csv

In [3]:
filename = "raw.csv"
df = pd.read_csv(f"../../data/resources/indicators/id_gove_01/{filename}")

In [4]:
column_value = "value_governance_raw_isolated"
melted_df = df.melt(
    id_vars=["ISO3", "Name"],
    var_name="Year",
    value_name=column_value
)




In [5]:
# Create a flag that indicates if value is real or mean
melted_df[f"no_{column_value}"] = melted_df[column_value].isna()
# Get mean by year and replace nan values by mean
mean_df = melted_df.groupby(["Year"])[[column_value]].mean().reset_index()
mean_df.rename(columns={column_value: "mean"}, inplace=True)
final_df = pd.merge(melted_df, mean_df, how="left", on="Year")
final_df.loc[final_df[column_value].isna(), column_value] = final_df["mean"]
del final_df["mean"]


In [6]:
melted_df

,ISO3,Name,Year,value_governance_raw_isolated,no_value_governance_raw_isolated
0,AFG,Afghanistan,1995,NaN,True
1,ALB,Albania,1995,NaN,True
2,DZA,Algeria,1995,NaN,True
3,AND,Andorra,1995,NaN,True
4,AGO,Angola,1995,NaN,True
...,...,...,...,...,...
4987,VEN,"Venezuela, Bolivarian Republic o",2020,-1.517866,False
4988,VNM,Viet Nam,2020,-0.073430,False
4989,YEM,Yemen,2020,-2.671682,False
4990,ZMB,Zambia,2020,-0.128189,False


In [8]:
# Save result to CSV
melted_filename = "melted_governance_noNormalized.csv"
final_df.to_csv(f"../../data/resources/indicators/id_gove_01/{melted_filename}", index=False)

## EXTRA: Let's convert governance csv but from raw and normalized

In [23]:
final_df_max_by_year = final_df.groupby(["Year"])[[column_value]].max().reset_index()
final_df_max_by_year.rename(columns={column_value: "max"}, inplace=True)

final_df_min_by_year = final_df.groupby(["Year"])[[column_value]].min().reset_index()
final_df_min_by_year.rename(columns={column_value: "min"}, inplace=True)

final_normalized_df = pd.merge(final_df, final_df_max_by_year, how="left", on="Year")
final_normalized_df = pd.merge(final_normalized_df, final_df_min_by_year, how="left", on="Year")
# final_normalized_df

# # # formula: (x - min) / (max - min)
# min = -2.5
# final_normalized_df["min"] = min
final_normalized_df["value_governance_normalized_isolated"] = 1 - ((final_normalized_df["value_governance_raw_isolated"] - final_normalized_df["min"]) / (final_normalized_df["max"] - final_normalized_df["min"]))
del final_normalized_df["value_governance_raw_isolated"]
del final_normalized_df["max"]
del final_normalized_df["min"]
final_normalized_df.rename(columns={"value_governance_normalized_isolated": "value_governance_raw_isolated"}, inplace=True)


# # To make more sense
# final_normalized_df["value_governance_raw_isolated_v2"] = 1 - final_normalized_df["value_governance_raw_isolated"]
final_normalized_df


,ISO3,Name,Year,no_value_governance_raw_isolated,value_governance_raw_isolated
0,AFG,Afghanistan,1995,True,NaN
1,ALB,Albania,1995,True,NaN
2,DZA,Algeria,1995,True,NaN
3,AND,Andorra,1995,True,NaN
4,AGO,Angola,1995,True,NaN
...,...,...,...,...,...
4987,VEN,"Venezuela, Bolivarian Republic o",2020,False,0.726031
4988,VNM,Viet Nam,2020,False,0.399939
4989,YEM,Yemen,2020,False,0.986514
4990,ZMB,Zambia,2020,False,0.412301


In [ ]:
final_df_min_by_year

In [ ]:
final_df_max_by_year

In [24]:
# Save result to CSV
melted_filename = "melted_governance.csv"
final_normalized_df.to_csv(f"../../data/resources/indicators/id_gove_01/{melted_filename}", index=False)